**Configuración**

In [1]:
# Constantes globales.
ESPACIOS_TOTALES = 50
PRECIO_HORA = 2000
PRECIO_CUARTO = 500
# Datos iniciales. (Usuario)
Administrador = {"Admin": "UdeA1803"}
# Diccionarios para guardar usuarios, vehículos, parqueo.
Usuarios = {}
Vehiculos = {}
Estacion = {}  # clave: espacio en el parquea, valor: datos de ingreso.
HistorialRetiros = []

# Usuario

In [14]:
# función para el registro de usuarios

#esta función se crea con el objetivo de almacenar las visitas historicas de los usuarios, haciendo más optimo el trabajo para calcular el descuentos(el cúal aún no hago), historial de uso, etc.
#dado el poco cupos de parqueaderos y la cantidad de persona. Lo justo es un sorteo de parqueadero por mes. Es decir, cuando una persona complete 30 parqueos, independientemente de que
#sea secuencial, el sistema le genere un descuento, rotación, o en su defecto retiro (es decir que vuelva a pedir al usuario que se inscriba.)

def RegistrarUsuario():
    print("\n*Registrar*")
    Documento = input("Por favor ingrese el documento: ")
# Verificar si el usuario ya está registrado
    if Documento in Usuarios:
        print(f"El documento{Documento}, ya se encuentra registrado en el sistema.")
        return
# registrar datos
    Nombre = input("Ingrese solo el nombre ;): ")
    Apellido = input("Ingrese el apellido: ")
    Placa = input("Ingrese la matricula del vehículo: ").upper()
# Convierte a mayúsculas automáticamente
# Guardar la información en el diccionario 'Usuarios'
    Usuarios[Documento] = {
        "nombre": Nombre,
        "apellido": Apellido,
        "placa": Placa
    }
    print("Usuario registrado éxitosamente :D.")


# Registrar vehiculos

In [15]:
from datetime import datetime

# Constantes y estructuras
EspaciosTotales = 50
Usuarios = {}
VehiculosParqueados = {}  # clave: placa, valor: datos
EstacionesDisponibles = list(range(1, EspaciosTotales + 1))

# Función para ingresar vehículo
def IngresarVehiculo():
    print("\n--- Ingreso de Vehículo ---")
    Documento = input("Documento de identidad del usuario: ")

# Validar si el usuario está registrado
    if Documento not in Usuarios:
        print("Usuario no registrado. Primero debe registrarlo.")
        return

# Localizar la placa asociada al documento
    Placa = Usuarios[Documento]["placa"]

# Verificar si el vehículo ya está estacionado
    if Placa in VehiculosParqueados:
        print("Este vehículo ya está estacionado.")
        return

# Verificar si hay estaciones disponibles
# promesa, encontrar la forma de que esta estacione acorde al estudiante y la cercania de su facultad.
    if not EstacionesDisponibles:
        print("Lo sentimos, todas nuestras estaciones están llenas.")
        return

# Asignar la primera estación libre
    EstacionAsignada = EstacionesDisponibles.pop(0)

# Registrar la hora actual
    HoraIngreso = datetime.now()


# Almacenar información del ingreso
    VehiculosParqueados[Placa] = {
        "documento": Documento,
        "nombre": Usuarios[Documento]["nombre"],
        "apellido": Usuarios[Documento]["apellido"],
        "estacion": EstacionAsignada,
        "hora_ingreso": HoraIngreso
    }

    print(f"Vehículo ingresado exitosamente. Estación asignada: {EstacionAsignada}\n")

# Ver usuarios

In [16]:
import pandas as pd

In [17]:
def VerUsuarios():
    print("\n--- Lista de Usuarios Registrados ---")
    if not Usuarios:
        print("No hay usuarios registrados.")
        return

    # Convertir el diccionario de usuarios en DataFrame
    df_usuarios = pd.DataFrame.from_dict(Usuarios, orient='index')
    df_usuarios.index.name = 'Documento'
    print(df_usuarios)

# Retirar vehiculo

In [18]:
def RetirarVehiculo():
    print("\n--- Retiro de Vehículo ---")
    Placa = input("Ingrese la placa del vehículo: ").upper()

    if Placa not in VehiculosParqueados:
        print("Vehículo no encontrado en el parqueadero.")
        return

    # Obtener datos del vehículo
    datos = VehiculosParqueados[Placa]
    HoraIngreso = datos["hora_ingreso"]
    HoraSalida = datetime.now()
    Duracion = HoraSalida - HoraIngreso
    Minutos = int(Duracion.total_seconds() // 60)

    # Cálculo del cobro
    if Minutos <= 30:
        Costo = 3000
    else:
        HorasCompletas = Minutos // 60
        Resto = Minutos % 60
        if Resto > 0:
            HorasCompletas += 1
        Costo = HorasCompletas * 3000

    # Mostrar factura
    print("\n----- FACTURA -----")
    print(f"Placa: {Placa}")
    print(f"Celda: {datos['estacion']}")
    print(f"Hora de ingreso: {HoraIngreso.strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"Hora de salida: {HoraSalida.strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"Tiempo total: {Minutos} minutos")
    print(f"Total a pagar: ${Costo}")
    print("--------------------\n")

    # Liberar celda
    EstacionesDisponibles.append(datos["estacion"])
    EstacionesDisponibles.sort()  # Para que siempre se asignen en orden

    # Registrar retiro en historial
    HistorialRetiros.append({
        "placa": Placa,
        "documento": datos["documento"],
        "nombre": datos["nombre"],
        "apellido": datos["apellido"],
        "celda": datos["estacion"],
        "ingreso": HoraIngreso.strftime('%Y-%m-%d %H:%M:%S'),
        "salida": HoraSalida.strftime('%Y-%m-%d %H:%M:%S'),
        "tiempo_min": Minutos,
        "costo": Costo
    })

    # Eliminar de vehículos activos
    del VehiculosParqueados[Placa]

# Menú de admin

**ingresos**

In [19]:
def VerIngresos():
    print("\n Ingresos Totales \n")
    if not HistorialRetiros:
        print("No hay retiros registrados aún.")
        return

    total_ingresos = sum(retiro["costo"] for retiro in HistorialRetiros)
    print(f"El total de ingresos generados es: ${total_ingresos}\n")

**Disponibilidad** de celdas.

In [20]:
def VerOcupacion():
    ocupadas = len(VehiculosParqueados)
    libres = len(EstacionesDisponibles)
    print("\n--- Disponibilidad del parqueadero ---")
    print(f"Celdas ocupadas: {ocupadas}")
    print(f"Celdas disponibles: {libres}\n")

**Resumen de vehículos estacionados**

In [21]:
def VerVehiculosParqueados():
    print("\n--- Vehículos Actualmente Parqueados ---")
    if not VehiculosParqueados:
        print("No hay vehículos en el parqueadero.")
        return

    df = pd.DataFrame.from_dict(VehiculosParqueados, orient='index')
    df = df[["nombre", "apellido", "documento", "estacion", "hora_ingreso"]]
    df["hora_ingreso"] = df["hora_ingreso"].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S'))
    print(df)

**historial de retiros**

In [10]:
def ExportarHistorialRetiros():
    if not HistorialRetiros:
        print("No hay datos en el historial de retiros para exportar.")
        return

    df = pd.DataFrame(HistorialRetiros)
    nombre_archivo = "historial_retiros.csv"
    df.to_csv(nombre_archivo, index=False)
    print(f"Historial de retiros exportado correctamente a '{nombre_archivo}'.")

**Menú de admin**

In [11]:
#Menú de admin
def MenuAdministrador():
    print("\n--- Estación Admin ---")
    usuario = input("Ingrese usuario: ")
    contraseña = input("Ingrese la contraseña: ")

    if usuario in Administrador and Administrador[usuario] == contraseña:
        print("Acceso permitido.\n")
        while True:
            print("--- Opciones de Administrador ---")
            print("1. Disponibilidad del parqueadero")
            print("2. Vehículos estacionados")
            print("3. Exportar historial de retiros")
            print("4. Ver Ingresos")
            print("5. Salir del administrador")
            opcion = input("Seleccione una opción: ")

            if opcion == "1":
                VerOcupacion()

            elif opcion == "2":
                VerVehiculosParqueados()

            elif opcion == "3":
                ExportarHistorialRetiros()

            elif opcion == "4":
                VerIngresos()

            elif opcion == "5":
                print("Saliendo del módulo de administración.\n")
                break
            else:
                print("Opción inválida. Intente de nuevo.\n")
    else:
        print("Usuario o contraseña incorrectos.\n")

# Menú General

In [22]:
def Menu():
    print("\n--- Bienvenido a PARQUEA UdeA ---")
    while True:
        print("1. Registrar")
        print("2. Ver usuarios")
        print("3. Ingresar vehículo")
        print("4. Retirar vehículo")
        print("5. Ingresar al módulo de administrador")
        print("6. Salir")

        opcion = input("Seleccione una opción: ")

        if opcion == "1":
            RegistrarUsuario()
        elif opcion == "2":
            VerUsuarios()
        elif opcion == "3":
            IngresarVehiculo()
        elif opcion == "4":
            RetirarVehiculo()
        elif opcion == "5":
            MenuAdministrador()
        elif opcion == "6":
            print("Gracias por usar Parquea. Hasta luego.")
            break
        else:
            print("Opción no válida. Intente nuevamente.")

# Ejecutar programa
Menu()


--- Bienvenido a PARQUEA UdeA ---
1. Registrar
2. Ver usuarios
3. Ingresar vehículo
4. Retirar vehículo
5. Ingresar al módulo de administrador
6. Salir
Seleccione una opción: 6
Gracias por usar Parquea. Hasta luego.


siguiente, crear modulo de administración.


¿que hará?
Pedirá usuario y contraseña.
Si son correctos, mostrará un menú con varias funciones:
Ver historial de retiros (la que ya tenemos).
Ver ocupación actual.
Ver usuarios registrados.
Ver vehículos en parqueadero.
Salir del módulo administrador.

pedir al usuario el destino en la universidad para generar la proximidad entre el parqueadero y el usuario.